In [61]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from copy import deepcopy
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc

In [62]:
# node class that each spot in the map will occupy
# cell location and goal_location are tuples representing index 
# of current cell location and goal cell locations
class Node:
    def __init__(self, parent, c2c, is_obstacle, cell_location):
        self.parent = parent
        self.c2c = c2c
        self.is_obstacle = is_obstacle
        self.cell_location = cell_location

In [63]:
def check_obstacle(w, h):

    # check to see if the cell is in the circle
    if(np.sqrt((w-200)**2 + (h-50)**2) <= 80):
        return True       

    # check triangle shape
    # divided into three shapes
    # left corner
    #      checking x bound                 checking below top slope                checking above bottom slope
    
    
    # top_slope_eq = 173.60759493670886 + (210-185)/(115-36)*w
    # bottom_slope_eq = 213.72340425531914 + (110-185)/(130-36)*w
    # if (w >= 36 and w <=80) and (h <=top_slope_eq) and (h >= bottom_slope_eq):
    #     return True
    # # now check to see if we are in the bounds of the concave parts
    # top_concave_slope_eq = 90 + (210-180)/(115-80)*w
    # bottom_concave_slope_eq = 309.3478260869565 + (100-185)/(105-36)*w
    # if (w >= 80 and w <= 115) and (h <=top_slope_eq) and (h >= top_concave_slope_eq):
    #     return True
    # if (w >= 80 and w <= 105) and (h >=bottom_slope_eq) and (h <= bottom_concave_slope_eq):
    #     return True


    return False

In [64]:
# create the board

def create_board(width, height):

    board = []
    for h in range(0, height):
        row = []
        for w in range(0, width):

            c2c = np.Infinity
            
            # is_obstacle = check_obstacle(w, h)
            is_obstacle = False

            new_node = Node(parent=None, c2c=c2c, is_obstacle=is_obstacle, cell_location=(w, h))
            row.append(new_node)

        board.append(row)

    return board

In [67]:
def create_color_map(board):

    color_map = np.zeros(shape=[250, 400, 3], dtype=np.uint8)

    for row in range(250):
        for col in range(400):
            
            if board[row][col].is_obstacle:
                color_map[row][col][0] = 255
                color_map[row][col][1] = 0
                color_map[row][col][2] = 0
            else:
                color_map[row][col][0] = 0
                color_map[row][col][1] = 0
                color_map[row][col][2] = 0
    
    return color_map


In [68]:
def update_color_map(curr_node, color_map,  brg_color):

    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]

    color_map[row][col][0] = brg_color[0]
    color_map[row][col][1] = brg_color[1]
    color_map[row][col][2] = brg_color[2]
    return color_map


In [69]:
def get_start_node(x, y, board):
    if (x >= 0 and x < 400) and (y >= 0 and y < 250):
        if not board[x][y].is_obstacle:
            return board[x][y]
        else:
            print('Board Location is in an obstacle. Choose another location')
            return
            
    print('Board Location Out of bounds. Choose another Location')
    return

In [70]:
def check_up(board, curr_node):
    
    x = curr_node.cell_location[0]
    y = curr_node.cell_location[1]
    
    # check out of bounds
    if y != 249:
        # check if obstacle
        new_node = board[x][y+1]
        if not new_node.is_obstacle:
            new_c2c = 1 + curr_node.c2c
            if new_c2c < new_node.c2c:
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [71]:
def check_down(board, curr_node):
    
    x = curr_node.cell_location[0]
    y = curr_node.cell_location[1]
    
    # check out of bounds
    if y != 0:
        # check if obstacle
        new_node = board[x][y-1]
        if not new_node.is_obstacle:
            new_c2c = 1 + curr_node.c2c
            if new_c2c < new_node.c2c:
                new_node.c2c = new_c2c
                new_node.parent = curr_node

            return new_node

    return None

In [72]:
def check_left(board, curr_node):
    
    x = curr_node.cell_location[0]
    y = curr_node.cell_location[1]
    
    # check out of bounds
    if x != 0:
        # check if obstacle

        new_node = board[x-1][y]
        if not new_node.is_obstacle:
            new_c2c = 1 + curr_node.c2c
            if new_c2c < new_node.c2c:
                new_c2c = new_c2c
                new_node.parent = curr_node
            
            return new_node

    return None

In [73]:
def check_right(board, curr_node):
    
    x = curr_node.cell_location[0]
    y = curr_node.cell_location[1]
    
    # check out of bounds
    if x != 399:
        # check if obstacle

        new_node = board[x+1][y]
        if not new_node.is_obstacle:
            new_c2c = 1 + curr_node.c2c
            if new_c2c < new_node.c2c:
                new_c2c = new_c2c
                new_node.parent = curr_node
            
            return new_node

    return None

In [74]:
def gen_next_nodes(board, curr_node):

    new_nodes = []

    new_nodes.append(check_up(board, curr_node))
    new_nodes.append(check_down(board, curr_node))
    new_nodes.append(check_left(board, curr_node))
    new_nodes.append(check_right(board, curr_node))

    return list(filter(lambda node: node is not None, new_nodes))

In [75]:
# construct the graph of nodes as well as an array of blank rbg values that will represent the board
# perform disjkstra using the node grah, but update the color array as we pop nodes

start_location = (0, 0)
width = 400
height = 250
goal_location = (100, 100)

board = create_board(width, height)
color_map = create_color_map(board)

start_node = get_start_node(start_location[0], start_location[1], board)
start_node.c2c = 0

open_nodes = [start_node]
closed_nodes = []

while len(open_nodes) > 0:
    # generate the colors of the current board and append it to the list
    # this will be a frame of an animation
    # color_maps.append(gen_color_map(board)
    open_nodes.sort(key=lambda x: x.c2c)
    curr_node = open_nodes.pop(0)
    closed_nodes.append(curr_node)


    x = curr_node.cell_location[0]
    y = curr_node.cell_location[1]
    print(f"Searching ({x},{y})")

    if curr_node.cell_location == goal_location:
        print('Found Solution')
        break
    else:
        next_possible_nodes = gen_next_nodes(board, curr_node)
        for node in next_possible_nodes:

            appendable = True

            for o_node in open_nodes:
                if o_node == node:
                    appendable = False
                    break
            if appendable:
                for c_node in closed_nodes:
                    if c_node == node:
                        appendable = False
                        break

            if appendable:
                open_nodes.append(node)



Searching (0,0)
Searching (1,0)
Searching (1,1)
Searching (0,1)
Searching (2,1)
Searching (2,0)
Searching (0,2)
Searching (2,2)
Searching (1,2)
Searching (3,0)
Searching (3,2)
Searching (3,1)
Searching (1,3)
Searching (3,3)
Searching (0,3)
Searching (2,3)
Searching (4,1)
Searching (4,3)
Searching (4,0)
Searching (4,2)
Searching (0,4)
Searching (2,4)
Searching (4,4)
Searching (1,4)
Searching (3,4)
Searching (5,0)
Searching (5,2)
Searching (5,4)
Searching (5,1)
Searching (5,3)
Searching (1,5)
Searching (3,5)
Searching (5,5)
Searching (0,5)
Searching (2,5)
Searching (4,5)
Searching (6,1)
Searching (6,3)
Searching (6,5)
Searching (6,0)
Searching (6,2)
Searching (6,4)
Searching (0,6)
Searching (2,6)
Searching (4,6)
Searching (6,6)
Searching (1,6)
Searching (3,6)
Searching (5,6)
Searching (7,0)
Searching (7,2)
Searching (7,4)
Searching (7,6)
Searching (7,1)
Searching (7,3)
Searching (7,5)
Searching (1,7)
Searching (3,7)
Searching (5,7)
Searching (7,7)
Searching (0,7)
Searching (2,7)
Searchin

In [ ]:
video_writer_fourcc = cv2.VideoWriter_fourcc(*"XVID")
outfile = cv2.VideoWriter('test.avi', video_writer_fourcc, 10, (500, 500))

for node in closed_nodes:
    color_map = update_color_map(node, color_map, (255, 255, 255))
    outfile.write(color_map)

outfile.release()


error: OpenCV(4.5.5) /io/opencv/modules/videoio/src/cap_ffmpeg.cpp:192: error: (-215:Assertion failed) image.depth() == CV_8U in function 'write'
